In [71]:
'''
--------------------------------------------------------------------------
DISCLAIMER:
This model is specifically trained on Jehovah's Witness reading material.
It does not generalize or represent the entire Shona language. Use with
this context in mind.
--------------------------------------------------------------------------
'''


"\n--------------------------------------------------------------------------\nDISCLAIMER:\nThis model is specifically trained on Jehovah's Witness reading material.\nIt does not generalize or represent the entire Shona language. Use with\nthis context in mind.\n--------------------------------------------------------------------------\n"

### 1. Dataset

In [72]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "".join([page.extract_text() for page in reader.pages])
    return text

shona_text = extract_text_from_pdf("shona_article.pdf")

In [73]:
import stopwordsiso as stopwords

# Get Shona stopwords
shona_stopwords = stopwords.stopwords("sn")

def remove_stopwords(text):
    # Split the text into words
    words = text.split()
    
    # Remove stopwords
    filtered_words = [word for word in words if word not in shona_stopwords]
    
    # Join words back into a string
    return ' '.join(filtered_words)

In [74]:
shona_text=remove_stopwords(shona_text)

### 2. Vocabulary

In [75]:
shona_text

'34567NOVEMBER 2023 NYAYA DZEKUDZIDZA JANUARY 8–FEBRUARY 4, 2024MUPROFITA ISAYA akataura kuti Jehovha ‘ndiMwari wecho- kwadi.’ Shoko rakashandurwa kuti “chokwadi” rinoreva kuti“ameni.” (Isa. 65:16, mashoko emuzasi) “Ameni” zvinorevakuti “ngazviitike,” kana kuti “chokwadi.” Shoko rekuti “ameni”parinoshandiswa muBhaibheri richitaura nezvaJehovha kanakuti Jesu, rinoita kuti tive nechokwadi chekuti zvavachataurazvichaitika. Saka zvairehwa naIsaya paaitaura nevaIsraeri nde-zvekuti: Zvese zvinotaurwa naJehovha zvinotoitika. Jehovhaakaratidza izvozvo nekuzadzisa zvese zvaakavimbisa. 2Tinogonawo here kuva nechokwadi chekuti zvatakavimbi- swa naJehovha nezveremangwana zvichazadzika? Anenge ma-kore 800 kubva panguva yakararama Isaya, muapostora Pauroakatsanangura kuti nei zvinovimbiswa naMwari zvichitoitika.Pauro akati: ‘Hazvibviri kuti Mwari areve nhema.’ (VaH. 6:18)Sezvo chitubu chisingakwanisi kubudisa mvura yakanaka pa-mwe chete nemvura ine munyu, Jehovha, Tsime rechokwadi, haakwanisi kutaur

In [76]:
import re
import string

def deep_clean(text):
    # Remove UTF-8 encoding characters
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove repetitive patterns of 'CRCR'
    text = re.sub(r'(CRCRCRCRCR)+', ' ', text)
    
    # Remove repetitive patterns of 'CRCR'
    text = re.sub(r'(CRCR)+', ' ', text)
    
    # Remove repetitive patterns of 'CRCR'
    text = re.sub(r'(CR)+', ' ', text)
    
    # Remove repetitive patterns of 'CRCR'
    text = re.sub(r'(CRCRCR)+', ' ', text)
    
    # Remove repetitive patterns of 'CRCR'
    text = re.sub(r'(CRCRCRCR)+', ' ', text)
    
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

shona_text = deep_clean(shona_text)


In [77]:
shona_text

'NOVEMBER NYAYA DZEKUDZIDZA JANUARY –FEBRUARY MUPROFITA ISAYA akataura kuti Jehovha ‘ndiMwari wecho kwadi’ Shoko rakashandurwa kuti “chokwadi” rinoreva kuti“ameni” Isa mashoko emuzasi “Ameni” zvinorevakuti “ngazviitike” kana kuti “chokwadi” Shoko rekuti “ameni”parinoshandiswa muBhaibheri richitaura nezvaJehovha kanakuti Jesu rinoita kuti tive nechokwadi chekuti zvavachataurazvichaitika Saka zvairehwa naIsaya paaitaura nevaIsraeri ndezvekuti Zvese zvinotaurwa naJehovha zvinotoitika Jehovhaakaratidza izvozvo nekuzadzisa zvese zvaakavimbisa Tinogonawo here kuva nechokwadi chekuti zvatakavimbi swa naJehovha nezveremangwana zvichazadzika Anenge makore kubva panguva yakararama Isaya muapostora Pauroakatsanangura kuti nei zvinovimbiswa naMwari zvichitoitikaPauro akati ‘Hazvibviri kuti Mwari areve nhema’ VaH Sezvo chitubu chisingakwanisi kubudisa mvura yakanaka pamwe chete nemvura ine munyu Jehovha Tsime rechokwadi haakwanisi kutaura nhema Saka tinogona kuvimba zvizere ne Mashoko emuprofita Is

In [78]:
from keras.preprocessing.text import Tokenizer

# Tokenize and preprocess
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts([shona_text])
total_words = len(tokenizer.word_index) + 1


### 3. Word Embeddings

In [79]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = [sentence.split() for sentence in shona_text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")


### 4. RNN Models

##### With its own embedding layer:

In [62]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(50, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(50))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


##### With pre-trained embeddings:

In [63]:
# Load the word2vec model
import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:
        # Word not present in gensim model, a zero embedding will be used for this word
        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(50, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(50))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 5. Training & Testing

In [65]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras

input_sequences = []
for line in shona_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
1241/1241 [==============================] - 53s 36ms/step - loss: 7.7557 - accuracy: 0.0516 - val_loss: 7.4904 - val_accuracy: 0.0575
Epoch 2/100
1241/1241 [==============================] - 43s 35ms/step - loss: 7.0674 - accuracy: 0.0622 - val_loss: 7.5043 - val_accuracy: 0.0676
Epoch 3/100
1241/1241 [==============================] - 45s 36ms/step - loss: 6.8125 - accuracy: 0.0756 - val_loss: 7.5247 - val_accuracy: 0.0792
Epoch 4/100
1241/1241 [==============================] - 43s 35ms/step - loss: 6.5630 - accuracy: 0.0874 - val_loss: 7.6041 - val_accuracy: 0.0777
Epoch 5/100
1241/1241 [==============================] - 45s 37ms/step - loss: 6.3278 - accuracy: 0.0949 - val_loss: 7.7080 - val_accuracy: 0.0846
Epoch 6/100
1241/1241 [==============================] - 50s 40ms/step - loss: 6.0781 - accuracy: 0.1032 - val_loss: 7.8142 - val_accuracy: 0.0885
Epoch 7/100
1241/1241 [==============================] - 43s 35ms/step - loss: 5.8182 - accuracy: 0.1135 - val_loss: 7

1241/1241 [==============================] - 45s 36ms/step - loss: 0.9477 - accuracy: 0.7980 - val_loss: 11.5121 - val_accuracy: 0.1267
Epoch 57/100
1241/1241 [==============================] - 44s 35ms/step - loss: 0.9162 - accuracy: 0.8045 - val_loss: 11.6159 - val_accuracy: 0.1265
Epoch 58/100
1241/1241 [==============================] - 44s 36ms/step - loss: 0.8973 - accuracy: 0.8085 - val_loss: 11.6474 - val_accuracy: 0.1279
Epoch 59/100
1241/1241 [==============================] - 43s 35ms/step - loss: 0.8715 - accuracy: 0.8126 - val_loss: 11.7387 - val_accuracy: 0.1250
Epoch 60/100
1241/1241 [==============================] - 44s 35ms/step - loss: 0.8580 - accuracy: 0.8130 - val_loss: 11.7396 - val_accuracy: 0.1270
Epoch 61/100
1241/1241 [==============================] - 44s 35ms/step - loss: 0.8315 - accuracy: 0.8193 - val_loss: 11.7878 - val_accuracy: 0.1281
Epoch 62/100
1241/1241 [==============================] - 44s 35ms/step - loss: 0.8102 - accuracy: 0.8244 - val_loss: 1

1241/1241 [==============================] - 29s 23ms/step - loss: 6.3995 - accuracy: 0.0735 - val_loss: 8.1312 - val_accuracy: 0.0736
Epoch 12/100
1241/1241 [==============================] - 28s 23ms/step - loss: 6.3153 - accuracy: 0.0739 - val_loss: 8.1685 - val_accuracy: 0.0736
Epoch 13/100
1241/1241 [==============================] - 29s 23ms/step - loss: 6.2371 - accuracy: 0.0741 - val_loss: 8.2427 - val_accuracy: 0.0733
Epoch 14/100
1241/1241 [==============================] - 28s 23ms/step - loss: 6.1616 - accuracy: 0.0750 - val_loss: 8.2784 - val_accuracy: 0.0711
Epoch 15/100
1241/1241 [==============================] - 29s 23ms/step - loss: 6.0921 - accuracy: 0.0760 - val_loss: 8.3445 - val_accuracy: 0.0731
Epoch 16/100
1241/1241 [==============================] - 28s 23ms/step - loss: 6.0249 - accuracy: 0.0751 - val_loss: 8.3648 - val_accuracy: 0.0763
Epoch 17/100
1241/1241 [==============================] - 29s 23ms/step - loss: 5.9657 - accuracy: 0.0759 - val_loss: 8.4420 

1241/1241 [==============================] - 28s 22ms/step - loss: 4.6048 - accuracy: 0.1373 - val_loss: 9.9696 - val_accuracy: 0.0811
Epoch 67/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5842 - accuracy: 0.1377 - val_loss: 10.0020 - val_accuracy: 0.0823
Epoch 68/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5759 - accuracy: 0.1393 - val_loss: 10.0074 - val_accuracy: 0.0801
Epoch 69/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5570 - accuracy: 0.1406 - val_loss: 10.0147 - val_accuracy: 0.0832
Epoch 70/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5471 - accuracy: 0.1438 - val_loss: 10.0352 - val_accuracy: 0.0787
Epoch 71/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5336 - accuracy: 0.1458 - val_loss: 10.0608 - val_accuracy: 0.0816
Epoch 72/100
1241/1241 [==============================] - 28s 23ms/step - loss: 4.5232 - accuracy: 0.1462 - val_loss: 10

### Model Evaluation

In [66]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")


Validation Loss for Model 1: 12.96580696105957
Validation Loss for Model 2: 10.458479881286621


In [67]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")


Saved the best model as best_model2.h5


C:\Users\tadiw\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [68]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")


Model 1 has 2127892 parameters.
Model 2 has 2127892 parameters.


In [69]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model1.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.
    
    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')
        
        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)
        
        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        
        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")


Please type five words in Shona: kubata patsinga yeruoko kuti anzwe
The next words might be: kuti kuti kuti
